# Machine Learning on Python - Naive Bayes 1

Welcome to the new notebook on Machine Learning Techniques on Python. In this notebook series, we will talk about Naive Bayes Classifier and how to use it on Python. Naive Bayes is a quite direct application of Bayes theorem and the mathematical details behind is not difficult. Let's get started.

## Mathematics behind Naive Bayes Classifier

Let's start with a quick recap: The Bayes Rule

$$P\left(H|E\right)=\frac{P\left(E|H\right)\cdot P\left(H\right)}{P\left(E\right)}$$

In the equation above, $P\left(H\:|\:E\right)$ is the probability of the hypothesis given that the evidence is true. $P\left(E|H\right)$ is the likelihood of the evidence given that the hypothesis is true, $P\left(H\right)$ is the prior probability of the hypothesis, and $P\left(E\right)$ is the prior probability that the evidence is true. I will not go into details about the Bayes Rule since I talked about it in my previous notebooks but you can find more about it in these videos [link1](https://www.youtube.com/watch?v=HZGCoVF3YvM) and [link2](https://www.youtube.com/watch?v=9wCnvr7Xw4E).

In the Naive Bayes Classifier, we will use the version of the Bayes rule in terms of features and target.

$$P\left(y|X\right)=\frac{P\left(X\:|\:y\right)\cdot P\left(y\right)}{P\left(x\right)}$$

Naive Bayes classifier assigns a posterior probability to an instance: $P\left(Y=y_j\:|\:X=x_i\right)$ and uses this probability to assign an instance to a class.

Naive Bayes Classifier assumes that the predictors $x_i$ are independent of each other, in other words, it assumes that every feature we have is independent of each other. That is an assumption because, in real-life data, it is highly likely that features depend on each other in determining the target. This assumption is actually the reason that the algorithm is named the "Naive" Bayes Classifier. Thanks to this assumption, we can rewrite the conditional probabilities $P\left(X\:|\:y\right)$ as follows:

$$P\left(X\:|\:y\right)\:\:\:=\:P\left(x_1\:|\:y\right)\:\cdot P\left(x_2\:|\:y\right)\:......\:P\left(x_n\:|\:y\right)\:$$

Let's rewrite the numerator of the Bayes theorem as well and get:


$$P\left(y_j|x\right)=\frac{\prod_{k=1}^p\left(P\left(x_k\:|\:y_j\right)\cdot \:P\left(y_j\right)\right)}{P\left(x\right)}$$

The denominator $P\left(x\right)$ does not depend on class, in other words, it is constant for all $y_j$. $P\left(x\right)$  acts as a scaling factor and ensures the posterior probability is between 0 and 1.  In the Naive Bayes Classifier, we are interested in assigning each feature to exactly one class, since the denominator does not depend on the class we can omit it and calculate the value of the numerator for each class and select the class with the maximal value to assign the feature. This is called the maximum posterior rule and the class with maximum posterior probability is called the maximum posteriori (MAP) class and it is calculated as follows:


$$y=argmax_{y_j}\prod_{k=1}^p\left(P\left(x_k\:|\:y_j\right)\cdot \:\:P\left(y_j\right)\right)$$

We are done with the general concept, let's now explore how to use the algorithm with Python, doing so we will also explore the different variants of the algorithm. The different naive Bayes classifiers differ mainly by the assumptions they make regarding the distribution.

**Additional Sources:** 

1. [Bayes’ Theorem and Naive Bayes Classifier](https://www.researchgate.net/publication/324933572_Bayes%27_Theorem_and_Naive_Bayes_Classifier?enrichId=rgreq-eb98ac66285b8c553b2bbaa1920edf73-XXX&enrichSource=Y292ZXJQYWdlOzMyNDkzMzU3MjtBUzo4MDQ4Mjk1MzQzNzU5MzZAMTU2ODg5NzcyMjA3Mg%3D%3D&el=1_x_2&_esc=publicationCoverPdf)






In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Gaussian Naive Bayes

There are different naive Bayes classifiers that generally differ mainly by their assumption of distribution. For instance, in Gaussian Naive Bayes the likelihood is assumed to be derived from Gaussian Distribution.

$$P\left(x_i\:|\:y\right)=\frac{1}{\sqrt{2\pi \sigma ^2_y}}exp\left(\frac{-\left(x_i-\mu _y\right)^2}{2\sigma \:^2_y}\right)$$

The parameters $\sigma _y$ and $\mu_y$ are calculated using Maximum Likelihood Estimation.

Let's use the algorithm.

For introducing how we can use the algorithm, I will use a toy dataset that we are already familiar with, Iris Setosa. Let's get the dataset using  `fetch_openml()` and then use `train_test_split()`.

In [2]:
from sklearn.naive_bayes import GaussianNB


In [3]:
setosa = fetch_openml('iris', version=1, as_frame=False)

X = setosa["data"]
y = setosa["target"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [5]:
gnb = GaussianNB()

In [6]:
model_gnb = gnb.fit(X_train, y_train)

In [7]:
y_pred=gnb.predict(X_test)

In [8]:
accuracy_score(y_test, y_pred)

0.9666666666666667

Accuracy score is quite high but it should be noted that this is only a toy dataset. Check the documentation for `GaussianNB()` [link](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)

## Multinomial Naive Bayes

Multinomial Naive Bayes is the implementation for multinomially distributed data. The distribution is parametrized by probability vector $\theta _y=\left(\theta _{y1},\theta _{y2},.....,\theta _{yn}\right)$ for each class $y$ where $n$ is the number of features and $\theta_y=P\left(x_i\:|\:y\right)$. $\theta _y$ is actually a smoother version of maximum likelihood and it is defined as follows:

$$\theta _{yi}=\frac{N_{yi}+a}{N_y+a\cdot n}$$

where $N_{yi}=\sum_{x\in T}\:\left(x_i\right)$ is the number of times feature $i$ appears in a sample of class $y$ in the training set T, $N_y=\sum^{^n}_{n=i}\:\left(N_{yi}\right)$  is the total count of all features for class $y$. The parameter $a$ is called smoothing prior and prevents zero probabilities. Setting  $a=1$ is called Laplace smoothing, while $a<1$  is called Lidstone smoothing.

In [9]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()

In [10]:
model_mnb = mnb.fit(X_train, y_train)

In [11]:
y_pred=mnb.predict(X_test)

In [12]:
accuracy_score(y_test, y_pred)

1.0

Documentation for `MultinomialNB()` : [link](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)

## Complement Naive Bayes

Complement Naive Bayes is an adaptation of multinomial naive Bayes for imbalanced datasets. CNB uses statistics from the complement of each class to compute the model’s weights. The procedure for calculating the weights is as follows:

$$w_{ci}=log\left(\theta _{ci}\right)=\frac{w_{ci}}{\sum_j\:\left|w_{cj}\right|}$$

where $$\theta _{ci}=\frac{a_i+\sum _{j:y_j\ne c}\left(d_{ij}\right)}{a+\sum _{j:y_j\ne \:c}\sum_k\left(d_{kj}\right)}\:$$ and $$a=\sum _i\left(a_i\right)$$

$a_i$ is again the smoothing parameter.

In [13]:
from sklearn.naive_bayes import ComplementNB

In [14]:
cnb=ComplementNB()

In [15]:
model_cnb = cnb.fit(X_train, y_train)

In [16]:
y_pred=cnb.predict(X_test)

In [17]:
accuracy_score(y_test, y_pred)

0.6666666666666666

You can find more mathematics behind Complement Naive Bayes in the paper [Tackling the poor assumptions of naive bayes text classifiers](https://people.csail.mit.edu/jrennie/papers/icml03-nb.pdf). Also check out the documentation for `ComplementNB()` [link](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html#sklearn.naive_bayes.ComplementNB)

## Bernoulli Naive Bayes

Bernoulli Naive Bayes implements the Naive Bayes algorithm for data that is distributed according to multivariate Bernoulli distribution. The algorithm uses the following learning rule for likelihood:

$$P\left(x_i\:|\:y\right)=P\left(x_i\:=1\:|\:y\right)x_i\:+\left(1-P\left(x_i\:=1\:|\:y\right)\right)\left(1-x_i\right)$$

Additionally, we can still use the algorithm for multiple labels, but in this case, the algorithm will binarize its input.

In [18]:
from sklearn.naive_bayes import BernoulliNB

In [19]:
bnb = BernoulliNB()

In [20]:
model_cnb = bnb.fit(X_train, y_train)

In [21]:
y_pred=bnb.predict(X_test)

In [22]:
accuracy_score(y_test, y_pred)

0.3333333333333333

Interestingly, the algorithm performed much worse that I expect it would.

For learning more about the mathematics behind Bernoulli Naive Bayes check out the paper [A. McCallum and K. Nigam (1998)](http://www.kamalnigam.com/papers/multinomial-aaaiws98.pdf) and [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB).

## Categorical Naive Bayes

Categorical Naive Bayes implements the naive Bayes algorithm for categorically distributed data and assumes each feature has its own categorical distribution.

The probability of category $t$ in feature $i$ given class $c$ is estimated as:


$$P\left(x_i=t\:|\:y=c;\:a\right)=\frac{N_{tic}+a}{N_c+an_i}$$  


where  $N_{tic}=\left|\left\{j\:\in J\:|\:x_{ij}\:=t,\:y_j=c\right\}\right|$ is the number of times category $t$ appears in the samples $x_i$ which belong to class $c$, $N_c=\left|\left\{j\:\in J\:|\:y_j=c\right\}\right|$ is the number of samples with class c, $a$ is a smoothing parameter and $n_i$ is the number of available categories of feature $i$.

In [23]:
from sklearn.naive_bayes import CategoricalNB

In [24]:
CategoricalNB=CategoricalNB()

In [25]:
model_cnb = CategoricalNB.fit(X_train, y_train)

In [26]:
y_pred=model_cnb.predict(X_test)

In [27]:
accuracy_score(y_test, y_pred)

0.9

An important thing that I wanted to note here, is `CategoricalNB()` assumes that each the sample matrix $X$ is encoded such that all categories for each feature $i$ are represented with numbers $0,1,.....,n_i-1\:$ where $n_i$ is the number of available categories in $i$. This may be a very important assumption while using `train_test_split()` which does not guarantee equally splitting the number of categories on train and test splits. Check out the [discussion](https://stackoverflow.com/questions/62011091/index-38-is-out-of-bounds-for-axis-1-with-size-38-sklearn)

Check out the documentation for `CategoricalNB()` [link](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html#sklearn.naive_bayes.CategoricalNB)

Additionally, I got a lot of help from Scikit-learn documentation for Naive Bayes while preparing this notebook. Check it out as well [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html#categorical-naive-bayes)

In the second notebook, we will be practicing the algorithms that we learnt in this one. Check out the second notebook.